<h1>Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Load-and-Define-Utils" data-toc-modified-id="Load-and-Define-Utils-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Load and Define Utils</a></span><ul class="toc-item"><li><span><a href="#Magics" data-toc-modified-id="Magics-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Magics</a></span></li><li><span><a href="#Easy-Widget" data-toc-modified-id="Easy-Widget-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Easy Widget</a></span></li></ul></li><li><span><a href="#Data" data-toc-modified-id="Data-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Data</a></span><ul class="toc-item"><li><span><a href="#Data-Source" data-toc-modified-id="Data-Source-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Data Source</a></span></li><li><span><a href="#Data-Convert" data-toc-modified-id="Data-Convert-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Data Convert</a></span></li><li><span><a href="#Data-Sink" data-toc-modified-id="Data-Sink-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>Data Sink</a></span></li><li><span><a href="#Test" data-toc-modified-id="Test-2.4"><span class="toc-item-num">2.4&nbsp;&nbsp;</span>Test</a></span></li><li><span><a href="#GST-Application" data-toc-modified-id="GST-Application-2.5"><span class="toc-item-num">2.5&nbsp;&nbsp;</span>GST Application</a></span></li></ul></li><li><span><a href="#JetRep" data-toc-modified-id="JetRep-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>JetRep</a></span><ul class="toc-item"><li><span><a href="#Inference-Process" data-toc-modified-id="Inference-Process-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Inference Process</a></span></li><li><span><a href="#Pre-Worker" data-toc-modified-id="Pre-Worker-3.2"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>Pre Worker</a></span></li><li><span><a href="#Post-Worker" data-toc-modified-id="Post-Worker-3.3"><span class="toc-item-num">3.3&nbsp;&nbsp;</span>Post Worker</a></span></li><li><span><a href="#Configure-Class" data-toc-modified-id="Configure-Class-3.4"><span class="toc-item-num">3.4&nbsp;&nbsp;</span>Configure Class</a></span></li><li><span><a href="#Control-Class" data-toc-modified-id="Control-Class-3.5"><span class="toc-item-num">3.5&nbsp;&nbsp;</span>Control Class</a></span></li></ul></li><li><span><a href="#References" data-toc-modified-id="References-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>References</a></span></li><li><span><a href="#FAQs" data-toc-modified-id="FAQs-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>FAQs</a></span></li></ul></div>

<div class="alert alert-warning">
<center>The Notebook Only Run On Jetson The Device</center>
</div>

In [1]:
notename = !uname -n
if 'nano' != notename[0]:
    raise RuntimeError('notebook server is not jetson device')

## Load and Define Utils

In [2]:
%reload_ext watermark
%reload_ext autoreload
%autoreload 2
# %watermark -p numpy,sklearn,pandas
# %watermark -p ipywidgets,cv2,PIL,matplotlib,plotly,netron
# %watermark -p torch,torchvision,torchaudio
# %watermark -p tensorflow,tensorboard,tflite
# %watermark -p onnx,tf2onnx,onnxruntime,tensorrt,tvm
# %matplotlib inline
%config InlineBackend.figure_format='retina'
# %config IPCompleter.use_jedi = False

from IPython.display import display, Markdown, HTML, IFrame, Image, Javascript
from IPython.core.magic import register_line_cell_magic, register_line_magic, register_cell_magic
# display(HTML('<style>.container { width:%d%% !important; }</style>' % 90))

import sys, os, io, logging, time, random, math
import json, base64, requests, shutil
import os.path as osp
import numpy as np
import argparse, shlex, signal
import threading, queue

argparse.ArgumentParser.exit = lambda *arg, **kwargs: _IGNORE_

def _IMPORT(x):
    try:
        x = x.strip()
        if x[0] == '/':
            with open(x) as fr:
                x = fr.read()
        elif 'github' in x or 'gitee' in x:
            if x.startswith('import '):
                x = x[7:]
            if x.startswith('https://'):
                x = x[8:]
            if not x.endswith('.py'):
                x = x + '.py'
            x = x.replace('blob/main/', '').replace('blob/master/', '')
            if x.startswith('raw.githubusercontent.com'):
                uri = 'https://' + x
                x = requests.get(uri)
                if x.status_code == 200:
                    x = x.text
            elif x.startswith('github.com'):
                uri = x.replace('github.com', 'raw.githubusercontent.com')
                mod = uri.split('/')
                for s in ['main', 'master']:
                    uri = 'https://' + '/'.join(mod[:3]) + s + '/'.join(mod[-3:])
                    x = requests.get(uri)
                    if x.status_code == 200:
                        x = x.text
                        break
            elif x.startswith('gitee.com'):
                mod = x.split('/')
                for s in ['/raw/main/', '/raw/master/']:
                    uri = 'https://' + '/'.join(mod[:3]) + s + '/'.join(mod[3:])
                    x = requests.get(uri)
                    if x.status_code == 200:
                        x = x.text
                        break
        exec(x, globals())
    except:
        pass

def _DIR(x, dumps=True, ret=True):
    attrs = sorted([y for y in dir(x) if not y.startswith('_')])
    result = '%s: %s' % (str(type(x))[8:-2], json.dumps(attrs) if dumps else attrs)
    if ret:
        return result
    print(result)
    
def get_logger(name, level=logging.DEBUG, filepath=None, console=True):
    logger = logging.getLogger(name)
    logger.setLevel(level)
    #  %(filename)s
    formatter = logging.Formatter('%(asctime)s - %(funcName)s:%(lineno)d - %(name)s - %(levelname)s - %(message)s')
    if console:
        console = logging.StreamHandler()
        console.setLevel(level)
        console.setFormatter(formatter)
        logger.addHandler(console)
    if filepath:
        filelog = logging.FileHandler(filepath)
        filelog.setLevel(level)
        filelog.setFormatter(formatter)
        logger.addHandler(filelog)
    return logger

import cv2

SCRIPT_DIR = '/home/nano/omega/jetson/scripts'
logger = get_logger('JetRep')

### Magics

In [3]:
@register_line_magic
def killall(line):
    pid_list=!ps -eo pid,command | grep $line  | grep -v 'grep' | cut -c1-6
    for pid in pid_list:
        pid = pid.strip()
        !kill -9 $pid

### Easy Widget

In [4]:
_IMPORT('https://gitee.com/qrsforever/nb_easy/easy_widget.py')

def switch_play_video(context, btnwid, oldval, newval, framecntwid, gstwid, frameimgwid):
    def _video_cap_loop():
        context.logger("Open GStreamer ... %s" % gstwid.value)
        try:
            cap = cv2.VideoCapture(gstwid.value, cv2.CAP_GSTREAMER)
            if cap.isOpened():
                context.logger('GStreamer is Opened!')
                count = 0
                while btnwid.value and count < framecntwid.value:
                    retval, frame = cap.read()
                    if retval:
                        frameimgwid.value = bytes(cv2.imencode(ext='.jpeg', img=frame)[1])
                    count += 1
                cap.release()
            btnwid.value = False
            context.logger('GStreamer is Stopped!')
        except Exception as err:
            context.logger('%s' % err)
    if newval:
        framecntwid.disabled = True
        btnwid.icon = 'stop-circle'
        btnwid.description = 'Stop'
        context.logger('Start Thread... ')
        threading.Thread(target=_video_cap_loop, args=()).start()
    else:
        framecntwid.disabled = False
        btnwid.icon = 'play'
        btnwid.description= 'Start'
        

def get_play_schema(gst):
     return {
        'type': 'page',
        'objs': [
            {
                'type': 'H',
                'objs': [
                    nbeasy_widget_togglebutton('switch_play', 'Start', icon='play'),
                    nbeasy_widget_int('frame_count', 'Frame Count', default=300, min_=100, max_=900, width=150),
                    nbeasy_widget_string('gst', 'GStreamer', gst, width='70%', readonly=True),
                ]
            },
            {
                'type': 'V',
                'objs': [
                    nbeasy_widget_image('frame_image', 'Image', '', format='jpeg')
                ],
                'align_items': 'center'
            },
            {
                'type': 'observe',
                'objs': [
                    {
                        'handler': switch_play_video,
                        'params': {
                            'source': 'switch_play',
                            'targets': [
                                'frame_count',
                                'gst',
                                'frame_image'
                            ]
                        }
                    },
                ]
            }
        ]
    }

# easy = nbeasy_schema_parse(get_play_schema(gst), debug=True);

## Data

In [5]:
import traitlets
import abc
import uuid
import datetime

In [6]:
class GSCom(traitlets.HasTraits):
    
    teename = traitlets.Unicode('')

    def __init__(self, parent, *args, **kwargs):
        super(GSCom, self).__init__(*args, **kwargs)
        if parent is not None:
            parent.children[self.name] = self
        self.parent = parent
        self.children = {}
            
    def gst_pipe(self):
        raise RuntimeError('abstract method called')
        
    def gst_str(self):
        gst = ' %s. ! queue ! ' % self.parent.teename if self.parent else ''
        gst += ' ! '.join(self.gst_pipe())
        for _, child in self.children.items():
            gst += child.gst_str()
        return gst 

In [32]:
isinstance(a, object)

True

In [31]:
a = list([10])

### Data Source

In [7]:
class Camera(traitlets.HasTraits):
    # 0: csi 1: usb
    device = traitlets.Int(min=0, max=1, default_value=0)

class DataSource(GSCom):
    teename = traitlets.Unicode('tee_raw')
    name = traitlets.Unicode()
    uuid = traitlets.Unicode('')
    width = traitlets.Int(default_value=640)
    height = traitlets.Int(default_value=480)
    framerate = traitlets.Int(default_value=30)
    
    number_buffers = traitlets.Int(default_value=-1)
    flip_method = traitlets.Int(default_value=0)
    
    def __init__(self, *args, **kwargs):
        super(DataSource, self).__init__(None, *args, **kwargs)
    
    @traitlets.default('uuid')
    def _default_uuid(self):
        return uuid.UUID(int=uuid.getnode()).hex[-12:]
   
    def gst_pipe(self):
        return [
            f'nvvidconv flip-method={self.flip_method}',
            f'"video/x-raw,width={self.width},height={self.height},format=BGRx"',
            f'textoverlay halignment=0 valignment=2 text={self.uuid}',
            f'clockoverlay halignment=2 time-format="%Y/%m/%d %H:%M:%S"',
            f'tee name={self.teename}'
        ]
    
class USBCamera(Camera, DataSource):
    name = traitlets.Unicode(default_value='usbcam')
    device = traitlets.Int(default_value=1)
    
        
    def gst_pipe(self):
        # io-mode = 2 (mmap)
        return [
            f'v4l2src name={self.name} device=/dev/video{self.device} io-mode=2 num-buffers={self.number_buffers}',
            f'"image/jpeg,width={self.width},height={self.height},framerate={self.framerate}/1,format=MJPG"',
            'nvjpegdec',
            '"video/x-raw"',
            'nvvidconv',
            f'"video/x-raw(memory:NVMM),width={self.width},height={self.height},format=NV12,framerate={self.framerate}/1"',
        ] + super().gst_pipe()
    

class CSICamera(Camera, DataSource):
    name = traitlets.Unicode(default_value='csicam')
    device = traitlets.Int(default_value=0)
    flip_method = traitlets.Int(default_value=6)
    
    def gst_pipe(self):
        return [
            f'nvarguscamerasrc name={self.name} sensor-id={self.device} num-buffers={self.number_buffers}',
            f'"video/x-raw(memory:NVMM),width={self.width},height={self.height},format=NV12,framerate={self.framerate}/1"',
        ] + super().gst_pipe()
 
CSICamera().gst_str(), '-'*60, USBCamera().gst_str()

('nvarguscamerasrc name=csicam sensor-id=0 num-buffers=-1 ! "video/x-raw(memory:NVMM),width=640,height=480,format=NV12,framerate=30/1" ! nvvidconv flip-method=6 ! "video/x-raw,width=640,height=480,format=BGRx" ! textoverlay halignment=0 valignment=2 text=48b02d5bcfa5 ! clockoverlay halignment=2 time-format="%Y/%m/%d %H:%M:%S" ! tee name=tee_raw',
 '------------------------------------------------------------',
 'v4l2src name=usbcam device=/dev/video1 io-mode=2 num-buffers=-1 ! "image/jpeg,width=640,height=480,framerate=30/1,format=MJPG" ! nvjpegdec ! "video/x-raw" ! nvvidconv ! "video/x-raw(memory:NVMM),width=640,height=480,format=NV12,framerate=30/1" ! nvvidconv flip-method=0 ! "video/x-raw,width=640,height=480,format=BGRx" ! textoverlay halignment=0 valignment=2 text=48b02d5bcfa5 ! clockoverlay halignment=2 time-format="%Y/%m/%d %H:%M:%S" ! tee name=tee_raw')

### Data Convert

In [8]:
class DataConvert(GSCom):
    name = traitlets.Unicode(allow_none=False)

    def gst_pipe(self):
        return [
            ' tee name=%s ' % self.teename
        ]

    
class H264CodecCvt(DataConvert):
    teename = traitlets.Unicode('tee_h264')
    name = traitlets.Unicode('h264', read_only=True)
    bitrate = traitlets.Int(default_value=2000000)
    
    def gst_pipe(self):
        # if self.parent.device == 1:
        #     videocvt, videoenc = 'videoconvert', 'x264enc tune=zerolatency'
        # else:
        #     videocvt, videoenc = 'nvvidconv', 'nvv4l2h264enc'
        return [
            'nvvidconv',
            f'nvv4l2h264enc bitrate={self.bitrate} maxperf-enable=1 profile=4 preset-level=1 iframeinterval=500 control-rate=1',
            'h264parse',
        ] + super().gst_pipe()
    
H264CodecCvt(CSICamera()).gst_str()

' tee_raw. ! queue ! nvvidconv ! nvv4l2h264enc bitrate=2000000 maxperf-enable=1 profile=4 preset-level=1 iframeinterval=500 control-rate=1 ! h264parse !  tee name=tee_h264 '

### Data Sink

In [9]:
class DataSink(GSCom):
    name = traitlets.Unicode(allow_none=False)
   
  
class ShareMemorySink(DataSink):
    name = traitlets.Unicode(default_value='shmsink')
    path = traitlets.Unicode(default_value='/tmp/gst_socket.shm')
    size = traitlets.Int(default_value=67108864, min=0, max=4294967295)
    
    def __init__(self, parent, *args, **kwargs):
        super(ShareMemorySink, self).__init__(parent, *args, **kwargs)
        # if os.path.exists(self.path):
        #     os.unlink(self.path)
    
    def gst_pipe(self):
        return [
            f'videoconvert',
            f'"video/x-raw, format=(string)BGR"',
            f'shmsink socket-path="{self.path}" shm-size={self.size} sync=true wait-for-connection=false'
        ]
    

class MultiFilesSink(DataSink):
    name = traitlets.Unicode(default_value='multifilessink')
    duration = traitlets.Int(default_value=600, min=1, max=3600)
    maxfiles = traitlets.Int(default_value=200, min=1, max=1000)
    location = traitlets.Unicode(default_value=f'/tmp/{datetime.datetime.now().strftime("%Y%m%d")}')
    
    def __init__(self, parent, *args, **kwargs):
        super(MultiFilesSink, self).__init__(parent, *args, **kwargs)
        os.makedirs(self.location, exist_ok=True)
    
    def gst_pipe(self):
        return [
            f'splitmuxsink muxer=qtmux max-size-time={self.duration * 1000000000} max-files={self.maxfiles} location="{self.location}/%03d.mp4"',
        ]


class SRSRtmpSink(DataSink):
    name = traitlets.Unicode(default_value='rtmpsink')
    server = traitlets.Unicode(default_value='127.0.0.1')
    port = traitlets.Int(default_value=1935)
    stream = traitlets.Unicode(allow_none=False)
    
    def __init__(self, parent, *args, **kwargs):
        super(SRSRtmpSink, self).__init__(parent, *args, **kwargs)
        
    def gst_pipe(self):
        return [
            'flvmux',
            f'rtmpsink location=rtmp://{self.server}:{self.port}/live/{self.stream}',
        ]
    
def test_gst(device=0, shm=True, mulfiles=True, rtmp=True):
    if device == 0:
        src = CSICamera()
    else:
        src = USBCamera()
    if shm:
        gst_sink_shm = ShareMemorySink(parent=src, path='/tmp/gst_repnet.shm', size=20000000)
    if mulfiles or rtmp:
        gst_cvt_h264 = H264CodecCvt(src)    
        if mulfiles:
            gst_sink_files = MultiFilesSink(parent=gst_cvt_h264, duration=300, maxfiles=10, location='/home/nano/omega/gst')
        if rtmp:
            gst_sink_rtmp = SRSRtmpSink(parent=gst_cvt_h264, server='172.16.0.35', stream='48b02d5bcfa5')
    return '%sgst-launch-1.0 -vvv -e ' % ('rm /tmp/gst_repnet.shm; ' if shm else '') + src.gst_str()

### Test

In [10]:
test_gst(device=0, shm=True, mulfiles=False, rtmp=True)

'rm /tmp/gst_repnet.shm; gst-launch-1.0 -vvv -e nvarguscamerasrc name=csicam sensor-id=0 num-buffers=-1 ! "video/x-raw(memory:NVMM),width=640,height=480,format=NV12,framerate=30/1" ! nvvidconv flip-method=6 ! "video/x-raw,width=640,height=480,format=BGRx" ! textoverlay halignment=0 valignment=2 text=48b02d5bcfa5 ! clockoverlay halignment=2 time-format="%Y/%m/%d %H:%M:%S" ! tee name=tee_raw tee_raw. ! queue ! videoconvert ! "video/x-raw, format=(string)BGR" ! shmsink socket-path="/tmp/gst_repnet.shm" shm-size=20000000 sync=true wait-for-connection=false tee_raw. ! queue ! nvvidconv ! nvv4l2h264enc bitrate=2000000 maxperf-enable=1 profile=4 preset-level=1 iframeinterval=500 control-rate=1 ! h264parse !  tee name=tee_h264  tee_h264. ! queue ! flvmux ! rtmpsink location=rtmp://172.16.0.35:1935/live/48b02d5bcfa5'

In [11]:
test_gst(device=1, shm=True, mulfiles=False, rtmp=False)

'rm /tmp/gst_repnet.shm; gst-launch-1.0 -vvv -e v4l2src name=usbcam device=/dev/video1 io-mode=2 num-buffers=-1 ! "image/jpeg,width=640,height=480,framerate=30/1,format=MJPG" ! nvjpegdec ! "video/x-raw" ! nvvidconv ! "video/x-raw(memory:NVMM),width=640,height=480,format=NV12,framerate=30/1" ! nvvidconv flip-method=0 ! "video/x-raw,width=640,height=480,format=BGRx" ! textoverlay halignment=0 valignment=2 text=48b02d5bcfa5 ! clockoverlay halignment=2 time-format="%Y/%m/%d %H:%M:%S" ! tee name=tee_raw tee_raw. ! queue ! videoconvert ! "video/x-raw, format=(string)BGR" ! shmsink socket-path="/tmp/gst_repnet.shm" shm-size=20000000 sync=true wait-for-connection=false'

In [12]:
test_gst(device=1, shm=False, mulfiles=False, rtmp=True)

'gst-launch-1.0 -vvv -e v4l2src name=usbcam device=/dev/video1 io-mode=2 num-buffers=-1 ! "image/jpeg,width=640,height=480,framerate=30/1,format=MJPG" ! nvjpegdec ! "video/x-raw" ! nvvidconv ! "video/x-raw(memory:NVMM),width=640,height=480,format=NV12,framerate=30/1" ! nvvidconv flip-method=0 ! "video/x-raw,width=640,height=480,format=BGRx" ! textoverlay halignment=0 valignment=2 text=48b02d5bcfa5 ! clockoverlay halignment=2 time-format="%Y/%m/%d %H:%M:%S" ! tee name=tee_raw tee_raw. ! queue ! nvvidconv ! nvv4l2h264enc bitrate=2000000 maxperf-enable=1 profile=4 preset-level=1 iframeinterval=500 control-rate=1 ! h264parse !  tee name=tee_h264  tee_h264. ! queue ! flvmux ! rtmpsink location=rtmp://172.16.0.35:1935/live/48b02d5bcfa5'

In [ ]:
%killall gst-launch

In [ ]:
%%bash --bg -s '{gst_source_csi.gst_str()}'

sudo rm -rf /tmp/gst_repnet.shm

bash -c "gst-launch-1.0 -e ${1}"

### GST Application

In [13]:
import traitlets
from traitlets.config.configurable import Configurable
from traitlets.config.application import Application
from traitlets import Bool, Unicode, Int, List, Dict, Enum
import gi
gi.require_version('Gst', '1.0')
from gi.repository import Gst, GObject

In [14]:
class GElement(Configurable):
    
    name = Unicode('')

    def __init__(self, pnode, *args, **kwargs):
        super(GElement, self).__init__(*args, **kwargs)
        if pnode is not None:
            pnode.children[self.name] = self
        self.pnode = pnode
        self.children = {}
            
    def gst_pipe(self):
        raise RuntimeError('abstract method called')
        
    def gst_str(self):
        gst = ' t_%s. ! queue ! ' % self.pnode.name if self.pnode else ''
        gst += ' ! '.join(self.gst_pipe())
        for _, child in self.children.items():
            gst += child.gst_str()
        return gst

class GDataSource(GElement):
    uuid = Unicode('')
    
    width = Int(default_value=640, help='Source video frame width').tag(config=True)
    height = Int(default_value=480, help='Source video frame height').tag(config=True)
    framerate = Int(default_value=30, help='Source video frame rate').tag(config=True)
    number_buffers = Int(default_value=-1, help='Number buffers of source video frames').tag(config=True)
    flip_method = Enum((0,1,2,3,4,5,6,7), default_value=0, help='Set the video flip methods').tag(config=True)
    
    def __init__(self, *args, **kwargs):
        super(GDataSource, self).__init__(None, *args, **kwargs)
    
    @traitlets.default('uuid')
    def _default_uuid(self):
        return uuid.UUID(int=uuid.getnode()).hex[-12:]
   
    def gst_pipe(self):
        return [
            f'nvvidconv flip-method={self.flip_method}',
            f'video/x-raw,width={self.width},height={self.height},format=BGRx',
            f'textoverlay halignment=0 valignment=2 text={self.uuid}',
            f'clockoverlay halignment=2 time-format="%Y/%m/%d %H:%M:%S"',
            f'tee name=t_{self.name}'
        ]
    
class USBCamera(GDataSource):
    name = Unicode(default_value='usbcam')
    device = Int(default_value=1)
    
    def gst_pipe(self):
        # io-mode = 2 (mmap)
        return [
            f'v4l2src name={self.name} device=/dev/video{self.device} io-mode=2 num-buffers={self.number_buffers}',
            f'image/jpeg,width={self.width},height={self.height},framerate={self.framerate}/1,format=MJPG',
            'nvjpegdec',
            'video/x-raw',
            'nvvidconv',
            f'video/x-raw(memory:NVMM),width={self.width},height={self.height},format=NV12,framerate={self.framerate}/1',
        ] + super().gst_pipe()
    
class CSICamera(GDataSource):
    name = Unicode(default_value='csicam')
    device = Int(default_value=0)
    flip_method = Enum((0,1,2,3,4,5,6,7), default_value=6, help='Set the video flip methods').tag(config=True)
    
    def gst_pipe(self):
        return [
            f'nvarguscamerasrc name={self.name} sensor-id={self.device} num-buffers={self.number_buffers}',
            f'video/x-raw(memory:NVMM),width={self.width},height={self.height},format=NV12,framerate={self.framerate}/1',
        ] + super().gst_pipe()

class GDataConvert(GElement):
    def gst_pipe(self):
        return [
            ' tee name=t_%s ' % self.name
        ]

class GH264CodecCvt(GDataConvert):
    name = Unicode(default_value='h264cvt')
    bitrate = Int(default_value=4000000, help='Set bitrate for v4l2 encode').tag(config=True)
    profile = Enum((0,2,4), default_value=4, help='Set profile for v4l2 encode').tag(config=True)
    preset_level = Enum((0,1,2,3,4), default_value=1, help='HW preset level for encode').tag(config=True)
    
    def gst_pipe(self):
        return [
            'nvvidconv',
            f'nvv4l2h264enc bitrate={self.bitrate} maxperf-enable=1 profile={self.profile} preset-level={self.preset_level} iframeinterval=500',
            'h264parse',
        ] + super().gst_pipe()    
    
    
class GDataSink(GElement):
    pass
   
class ShareMemorySink(GDataSink):
    name = Unicode(default_value='shmsink')
    path = Unicode(default_value='/tmp/gst_socket.shm', help='The path to the control socket(shared memory).').tag(config=True)
    size = Int(default_value=10000000, min=0, max=4294967295, help='Size of the shared memory area').tag(config=True)
    
    @traitlets.validate('path')
    def _check_socket_path(self, proposal):
        path = proposal['value']
        if not os.access(os.path.dirname(path), os.X_OK):
            raise RuntimeError(f'Cannot operate {path} permission!')
        if os.path.exists(path):
            os.unlink(path)
        return path
    
    def gst_pipe(self):
        return [
            f'videoconvert',
            f'video/x-raw, format=(string)BGR',
            f'shmsink socket-path="{self.path}" shm-size={self.size} sync=true wait-for-connection=false'
        ]

class MultiFilesSink(GDataSink):
    name = Unicode(default_value='multifilessink')
    duration = Int(default_value=600, min=1, max=3600, help='Max. amount of time per file').tag(config=True)
    maxfiles = Int(default_value=200, min=1, max=1000, help='Maximum number of files to keep on disk.').tag(config=True)
    location = Unicode(default_value='', allow_none=False, help='Format string pattern for the location of the files to write').tag(config=True)
    
    @traitlets.validate('location')
    def _check_location(self, proposal):
        path = proposal['value']
        if not os.access(os.path.dirname(path), os.X_OK):
            raise RuntimeError(f'Cannot operate {path} permission!')
        os.makedirs(path, exist_ok=True)
        return path
    
    def gst_pipe(self):
        return [
            f'splitmuxsink muxer=qtmux max-size-time={self.duration * 1000000000} max-files={self.maxfiles} location="{self.location}/%03d.mp4"',
        ]

class SRSRtmpSink(GDataSink):
    name = Unicode(default_value='rtmpsink')
    server = Unicode(default_value='0.0.0.0', help='RTMP server address').tag(config=True)
    port = Int(default_value=1935, help='RTMP server port').tag(config=True)
    stream = Unicode(allow_none=False, help='RTMP server stream').tag(config=True)
    max_lateness = Int(default_value=3, help='Maximum number of seconds that a buffer can be late before it is dropped').tag(config=True)
    
    def gst_pipe(self):
        return [
            'flvmux',
            f'rtmpsink max-lateness={self.max_lateness * 1000000000} location=rtmp://{self.server}:{self.port}/live/{self.stream}',
        ]

# GObject.threads_init()
# Gst.init(None)

class GstPipelineApp(Application):
    """
    This is a gstream pipleline application.
    """
    name = Unicode('GstPipeline')
    description = Unicode(__doc__)
    config_file = Unicode('', help='Load this config file').tag(config=True)
    log_file = Unicode('/tmp/gstpipe.log', help='Write log to file ').tag(config=True)

    camera = Enum((0,1,'CSI', 'USB'), default_value=1, help='Set the video camera device').tag(config=True)
    shmsink = Bool(default_value=True, help='Use shared memory sink source').tag(config=True)
    rtmpsink = Bool(default_value=False, help='Sends FLV content to a server via RTMP').tag(config=True)
    filesink = Bool(default_value=False, help='Write stream to files').tag(config=True)

    classes = List([USBCamera, CSICamera, GH264CodecCvt, ShareMemorySink, MultiFilesSink, SRSRtmpSink])

    aliases = Dict(
        dict(
            c='GstPipelineApp.config_file'
        )
    )

    flags = Dict(
        dict(
            debug=({'GstPipelineApp': {'log_level': 10}}, 'Set loglevel to DEBUG'),
            csi=({'GstPipelineApp': {'camera': 0}}, 'Use the csi camera'),
            usb=({'GstPipelineApp': {'camera': 1}}, 'Use the usb camera'),
            rtmpsink=({'GstPipelineApp': {'rtmpsink': True}}, 'Videos to RTMP Server'),
            filesink=({'GstPipelineApp': {'filesink': True}}, 'Videos to OS disks')
        )
    )

    @traitlets.default('log')
    def _log_default(self):
        log = logging.getLogger(self.__class__.__name__)
        # log.setLevel(self.log_level)
        log.setLevel(logging.DEBUG)
        formatter = self._log_formatter_cls(fmt=self.log_format, datefmt=self.log_datefmt)
        console = logging.StreamHandler()
        console.setFormatter(formatter)
        filelog = RotatingFileHandler(
                self.log_file, mode='a', maxBytes=5*1024*1024, backupCount=2, encoding=None, delay=0)
        filelog.setFormatter(formatter)
        log.addHandler(console)
        log.addHandler(filelog)
        return log

    def initialize(self, argv=None):
        self.parse_command_line(argv)
        if self.config_file:
            self.load_config_file(self.config_file)

    def setup(self):
        self.log.info('Setup components')
        if self.camera in ('1', 'USB'):
            cam = USBCamera(parent=self)
        else:
            cam = CSICamera(parent=self)

        if self.shmsink:
            _ = ShareMemorySink(cam, parent=self) # noqa

        if self.rtmpsink or self.filesink:
            h264cvt = GH264CodecCvt(cam, parent=self)
            if self.rtmpsink:
                _ = SRSRtmpSink(h264cvt, parent=self)
            if self.filesink:
                _ = MultiFilesSink(h264cvt, parent=self) # noqa

        self.gst_str = cam.gst_str()
        self.log.info(f'Gst string: {self.gst_str}')

    def start(self):
        self.log.info('Starting...')

        self.start_launch()

    def stop(self):
        self.log.info('Stoped.')
        self.stop_launch()

    def start_launch(self):
        self.log.info('Start gst..')
        self.gst_pipe = Gst.parse_launch(self.gst_str)
        bus = self.gst_pipe.get_bus()
        bus.add_signal_watch()
        bus.connect('message', self.on_message, self.mainloop)
        self.gst_pipe.set_state(Gst.State.READY)
        self.gst_pipe.set_state(Gst.State.PAUSED)
        self.gst_pipe.set_state(Gst.State.PLAYING)

    def stop_launch(self):
        self.gst_pipe.set_state(Gst.State.NULL)

    def on_message(self, bus: Gst.Bus, msg: Gst.Message, loop: GObject.MainLoop):
        self.log.info(f'Message type: {msg.type}')
        t = msg.type
        if t == Gst.MessageType.EOS:
            self.log.info('Stream EOS')
            loop.quit()
        elif t == Gst.MessageType.ERROR:
            self.log.error('{}: {}'.format(msg.parse_error()))
            loop.quit()
        elif t == Gst.MessageType.WARNING:
            self.log.warning('{}: {}'.format(msg.parse_warning()))
        return True
            
    def run(self, argv=None):
        try:
            self.mainloop = GObject.MainLoop()
            self.initialize(argv)
            self.setup()
            self.start()
            self.mainloop.run()
        except Exception:
            self.mainloop.quit()
            self.log.error(traceback.format_exc(limit=6))

/usr/lib/python3/dist-packages/ipykernel_launcher.py:246: PyGIDeprecationWarning: GObject.MainLoop is deprecated; use GLib.MainLoop instead


In [15]:
%%writefile /home/nano/omega/jetson/jetgst.json
{
    "GstPipelineApp": {
        "log_level": 10,
        "log_datefmt": "%Y-%m-%d %H:%M:%S",
        "log_format": "%(asctime)s %(funcName)s:%(lineno)d %(name)s %(levelname)s %(message)s",
        "camera": "USB"
    },
    "USBCamera": {
        "number_buffers": -1,
        "width": 640,
        "height": 480,
        "framerate": 30,
        "flip_method": 0
    },
    "CSICamera": {
        "width": 640,
        "height": 480,
        "framerate": 30,
        "flip_method": 0
    },
    "GH264CodecCvt": {
        "bitrate": 5000000
    },
    "ShareMemorySink": {
        "path": "/tmp/gst_repnet.shm",
        "size": 10000000
    },
    "MultiFilesSink": {
        "location": "/home/nano/omega/gst" 
    },
    "SRSRtmpSink": {
        "server": "0.0.0.0",
        "port": 1935,
        "stream": "repnet_raw"
    }
}

Overwriting /home/nano/omega/jetson/jetgst.json


In [16]:
%%writefile /home/nano/omega/jetson/install_jetgst_service.sh
#!/bin/bash

USER=$(whoami)
CUR_DIR=$(cd $(dirname ${BASH_SOURCE[0]}); pwd)
TOP_DIR=$(dirname $CUR_DIR)
DST_DIR=/etc/systemd/system/

SERVICE=jetgst.service

if [[ 0 != $(id -u) ]]
then
    echo "Use root execute!!!"
    exit 0
fi

cat > $TOP_DIR/etc/systemd/$SERVICE <<EOF
[Unit]
    Description=Gst pipeline start launch
    Documentation=http://jetrep.hzcsai.com
    After=srs.service multi-user.target

[Service]
    Type=simple
    User=$USER
    Group=$USER
    UMask=0000
    WorkingDirectory=$TOP_DIR
    EnvironmentFile=$TOP_DIR/etc/jetgst.env
    Restart=always
    RestartSec=5
    ExecStartPre=/bin/systemctl restart nvargus-daemon
    ExecStart=/usr/bin/python3 jetrep/stream -c etc/jetgst.json
    TimeoutStartSec=10
    TimeoutStopSec=5
    StandardOutput=syslog
    StandardError=syslog

[Install]
    WantedBy=multi-user.target
EOF

cp $TOP_DIR/etc/systemd/$SERVICE $DST_DIR
systemctl daemon-reload
systemctl enable $SERVICE
systemctl restart $SERVICE
systemctl status $SERVICE
echo "-------------------------------"
echo ""
echo "journalctl -u $SERVICE -f"
echo ""
echo "-------------------------------"


Overwriting /home/nano/omega/jetson/install_jetgst_service.sh


```sh
sudo cp jetgst.service /etc/systemd/system/
sudo systemctl daemon-reload
sudo systemctl enable jetgst.service
sudo systemctl start jetgst.service
sudo systemctl stop jetgst.service
systemctl status jetgst.service
journalctl -u jetgst.service -f
```

## JetRep

In [39]:
from multiprocessing import Queue, Process
from subprocess import Popen, PIPE, check_output, TimeoutExpired
from threading import Thread
import psutil
import queue

def get_pids(proname=None, cmdstr=None):
    if proname:
        try:
            return list(map(int, check_output(["pidof", proname]).split()))
        except:
            return []
    if cmdstr:
        pids = []
        for proc in psutil.process_iter():
            if cmdstr in ' '.join(proc.cmdline()):
                pids.append(proc.pid)
        return pids
    
# get_pids('gst-launch-1.0')

def kill_tree(pid, including_parent=True):    
    parent = psutil.Process(pid)
    children = parent.children(recursive=True)
    for child in children:
        print(child.pid)
        child.kill()
    gone, still_alive = psutil.wait_procs(children, timeout=5)
    if including_parent:
        parent.kill()

def sigmoid(x):
    return 1 / (1 + np.exp(-x))

def softmax(x, axis=None):
    e_x = np.exp(x - np.max(x, axis=axis, keepdims=True))
    return e_x / np.sum(e_x, axis=axis, keepdims=True)

In [46]:
import gevent

In [ ]:
gevent.wait()

### Inference Process

In [18]:
def repnet_inference_process(model_path, batch_queue, result_queue, status_queue, timeout):
    import tensorrt as trt
    import queue
    sys.path.append('/usr/src/tensorrt/samples/python')
    import common
    
    status_queue.put((100, 'starting'))
    try:
        with open(model_path, "rb") as f, \
            trt.Runtime(trt.Logger()) as runtime, \
            runtime.deserialize_cuda_engine(f.read()) as engine, \
            engine.create_execution_context() as context:

            inputs, outputs, bindings, stream = common.allocate_buffers(engine)

            status_queue.put((101, 'running'))
            while True:
                try:
                    token, np_frames = batch_queue.get(timeout=timeout)
                    if token < 0:
                        status_queue.put((0, 'quit'))
                        break
                except queue.Empty:
                    continue
                except Exception as err:
                    status_queue.put((-1, f'err: {err}'))
                    break

                np_frames = np_frames.astype(np.float32)
                np_frames -= 127.5
                np_frames /= 127.5
                inputs[0].host = np.reshape(np_frames, (1, 64, 112, 112, 3))

                trt_outputs = common.do_inference_v2(context, bindings=bindings, inputs=inputs, outputs=outputs, stream=stream)
                trt_outputs = [output.reshape(shape) for output, shape in zip(trt_outputs, ((-1, 1), (-1, 32)))]

                result_queue.put((token, trt_outputs[0].copy(), trt_outputs[1].copy()))
    except Exception as err:
        status_queue.put((-2, f'err: {err}'))

### Pre Worker

In [19]:
def prerep_process_worker(app, shm_path):
    app.logger.debug("Start prerep_process_worker...")
    gst_str = ' ! '.join([
        f'shmsrc socket-path={shm_path}',
        f'video/x-raw, format=BGR,width={app.frame_size[0]},height={app.frame_size[1]},framerate={app.frame_rate}/1',
        'appsink drop=1'
    ])
    try:
        app.logger.debug(f'Open cammera: [{gst_str}]')
        cap = cv2.VideoCapture(gst_str, cv2.CAP_GSTREAMER)
        if not cap.isOpened():
            raise RuntimeError('Could not open camera.')
        retval, _ = cap.read()
        if not retval:
            raise RuntimeError('Could not read image from camera.')

        frame_size = (
            int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)),
            int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
        )
        frame_rate = int(cap.get(cv2.CAP_PROP_FPS))
        app.logger.debug(f'Frame info: {frame_rate} {frame_size}')
    except:
        raise RuntimeError('Could not initialize camera.  Please see error trace.')
        
    pre_frame = None
    keep_flag = 1
    fbat = None
    while app.prerep:
        try:
            # app.logger.info("read")
            retval, frame_bgr = cap.read()
            if not retval:
                break
            if fbat is None:
                fbat = app.new_fbatch()
                writer = cv2.VideoWriter(
                    fbat.raw_frames_path,
                    cv2.VideoWriter_fourcc(*'mp4v'), frame_rate, frame_size
                )
        except Exception as err:
            app.logger.error(f'Err: {err}')
            break
        fbat.raw_frames_count += 1
        writer.write(frame_bgr)

        if app._black_box:
            black_x1, black_y1, black_x2, black_y2 = app._black_box
            frame_bgr[black_y1:black_y2, black_x1:black_x2, :] = 0

        if app._focus_box:
            focus_x1, focus_y1, focus_x2, focus_y2 = app._focus_box
            frame_bgr = frame_bgr[focus_y1:focus_y2, focus_x1:focus_x2, :]

        if app._area_thresh > 0:
            frame_gray = cv2.cvtColor(frame_bgr, cv2.COLOR_BGR2GRAY)
            if pre_frame is not None:
                frame_tmp = cv2.absdiff(frame_gray, pre_frame)
                frame_tmp = cv2.threshold(frame_tmp, 20, 255, cv2.THRESH_BINARY)[1]
                frame_tmp = cv2.dilate(frame_tmp, None, iterations=2)
                contours, _ = cv2.findContours(frame_tmp, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
                if len(contours) > 0:
                    for contour in contours:
                        if cv2.contourArea(contour) > app._area_thresh:
                            keep_flag += 1
                            break
            pre_frame = frame_gray
        else:
            keep_flag += 1
            
        # app.logger.info(fbat)

        if keep_flag % app._stride == 0 or fbat.raw_frames_count > app._max_raw_frames:
            frame_bgr = cv2.resize(frame_bgr, (112, 112))
            frame_rgb = cv2.cvtColor(frame_bgr, cv2.COLOR_BGR2RGB)
            if fbat.raw_frames_count > app._max_raw_frames:
                diff = 64 - len(fbat.selected_indices) 
                for _ in range(diff):
                    fbat.selected_indices.append(fbat.raw_frames_count - 1)
                    fbat.inputs.append(frame_rgb)
                app.logger.warning(f'May be still frames is to many, diff[{diff}].')
            else:
                fbat.selected_indices.append(fbat.raw_frames_count - 1)
                fbat.inputs.append(frame_rgb)
            if len(fbat.selected_indices) == 64:
                # app.logger.info(fbat)
                app.put_inputs_data(fbat)
                writer.release()
                fbat = None
            
            keep_flag = 1
            
    if cap.isOpened():        
        cap.release()
    app.logger.warning('PreRep process worker end!')

### Post Worker

In [20]:
def postrep_process_worker(app):
    app.logger.debug("Start postrep_process_worker...")
    
    gst_str = ' ! '.join([
        'appsrc',
        'videoconvert',
        'video/x-raw,format=BGRx',
        'nvvidconv',
        'nvv4l2h264enc bitrate=4000000',
        'video/x-h264,stream-format=(string)byte-stream,alignment=(string)au',
        'h264parse',
        'queue',
        'flvmux streamable=true name=mux',
        'rtmpsink max-lateness=500000 location=rtmp://0.0.0.0:1935/live/post_test'
    ])

    writer = cv2.VideoWriter(gst_str, 0, app.frame_rate, app.frame_size)
    sumcount = 0
    while app.postrep:
        try:
            fbat, within_scores, period_scores = app.get_outputs_data()
            if fbat is None:
                continue
        except queue.Empty:
            continue
        except Exception as err:
            logger.error(f'Err: {err}')
            break
        
        per_frame_periods = np.argmax(period_scores, axis=-1) + 1
        conf_pred_periods = np.max(softmax(period_scores, axis=-1), axis=-1)
        conf_pred_periods = np.where(per_frame_periods < 3, 0.0, conf_pred_periods)
            
        within_period_scores = sigmoid(within_scores)[:, 0]
        within_period_scores = np.sqrt(within_period_scores * conf_pred_periods)
        within_period_binary = np.asarray(within_period_scores > 0.5)
        
        per_frame_counts = within_period_binary * np.where(per_frame_periods < 3, 0.0, 1 / per_frame_periods)

        logger.debug(f'{fbat.raw_frames_path}: {sum(per_frame_counts)}')
        frame_counts = [0] * fbat.raw_frames_count
        s = 0
        for i, t in enumerate(fbat.selected_indices):
            for j in range(s, t):
                frame_counts[j] = per_frame_counts[i] / (t - s)
            s = t
        if os.path.exists(fbat.raw_frames_path):
            cap = cv2.VideoCapture(fbat.raw_frames_path)
            frame_counts = np.cumsum(frame_counts) + sumcount
            for c in frame_counts:
                retval, frame_bgr = cap.read()
                if not retval:
                    break
                cv2.putText(frame_bgr, f'%.3f' % c, (50, 100), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)
                writer.write(frame_bgr)
            else:
                sumcount = c
            cap.release()
            os.unlink(fbat.raw_frames_path)
    writer.release()    
    app.logger.warning('PostRep process worker end!')

###  Configure Class

In [21]:
class ProcessContext(traitlets.HasTraits):
    
    frame_size = traitlets.Tuple(traitlets.Int(), traitlets.Int())
    focus_box = traitlets.List(trait=traitlets.Float(min=0., max=1.), default_value=[0.0, 0.1, 1.0, 1.0], minlen=4, maxlen=4)
    black_box = traitlets.List(trait=traitlets.Float(min=0., max=1.), default_value=[0.0, 0.0, 1.0, 1.0], minlen=4, maxlen=4)
    area_rate = traitlets.Float(default_value=0.002, min=0.0, max=0.05)
    strides = traitlets.List(trait=traitlets.Int(), default_value=[4], minlen=1, maxlen=3)
    
    F = 64
    K = F * 10
    
    def __init__(self, logger, mdata_in, mdata_out, *args, **kwargs):
        super(ProcessContext, self).__init__(*args, **kwargs)
        
        self.logger = logger
        self._focus_box = None
        self._black_box = None
        self._stride = 4
        self._max_raw_frames = self._stride * self.K
        self._area_thresh = 0
        self._fbatches = {}
        
        self.frame_size = (640, 480)
        self.frame_rate = 30
        
        self.mdata_in = mdata_in
        self.mdata_out = mdata_out

        
    class FBatch(object):
        tmp_dir = '/home/nano/omega/gst'
        def __init__(self):
            self.token = int(time.time() * 1000)
            self.raw_frames_count = 0
            self.raw_frames_path = f'{self.tmp_dir}/{self.token}.mp4'
            self.inputs = []
            self.selected_indices = []
            
        def __str__(self):
            return 'token: {}, raw_frames_count: {}, raw_frames_path: {}, selected_indices: {}'.format(
                self.token,
                self.raw_frames_count,
                self.raw_frames_path,
                self.selected_indices
            )
        
        
    def new_fbatch(self):
        fbat = self.FBatch()
        return fbat
    
    def put_inputs_data(self, fbat):
        self._fbatches[fbat.token] = fbat
        self.mdata_in.put((fbat.token, np.asarray(fbat.inputs)))
        
    def get_outputs_data(self):
        token, within_scores, period_scores = self.mdata_out.get(timeout=3)
        fbat = None
        if token in self._fbatches:
            fbat = self._fbatches.pop(token)
        return fbat, within_scores, period_scores

    @staticmethod
    def calc_rect_box(w, h, box):
        return [int(w * box[0]), int(h * box[1]), int(w * box[2]), int(h * box[3])]
        
    @traitlets.observe('frame_size')
    def _on_frame_size(self, change):   
        if self._focus_box:
            self._focus_box = self.calc_rect_box(change['new'][0], change['new'][1], self.focus_box)
        if self._black_box:
            self._black_box = self.calc_rect_box(change['new'][0], change['new'][1], self.black_box)
        if self._focus_box:
            x1, y1, x2, y2 = self._focus_box 
            self._area_thresh = int((x2 - x1) * (y2 - y1) * self.area_rate)
        else:
            self._area_thresh = int(change['new'][0] * change['new'][1] * self.area_rate)
        
    @traitlets.observe('focus_box')
    def _on_focus_box(self, change):
        if change['new'] == [0., 0., 1., 1.]:
            self._focus_box = None
            return
        self._focus_box = self.calc_rect_box(self.frame_size[0], self.frame_size[1], change['new'])
        
    @traitlets.observe('black_box')
    def _on_black_box(self, change):
        if change['new'] == [0., 0., 0., 0.] or change['new'] == [1., 1., 1., 1.]:
            self._black_box = None
            return
        self._black_box = self.calc_rect_box(self.frame_size[0], self.frame_size[1], change['new'])
        
    @traitlets.observe('area_rate')
    def _on_area_rate(self, change):
        if self._focus_box:
            x1, y1, x2, y2 = self._focus_box 
            self._area_thresh = (x2 - x1) * (y2 - y1) * change['new']
            return
        self._area_thresh = int(self.frame_size[0] * self.frame_size[1] * change['new'])
        
    @traitlets.observe('strides')
    def _on_strides(self, change):
        # TODO only support one stride
        self._stride = change['new'][0]
        self._max_raw_frames = self._stride * sefl.K
        
    @traitlets.validate('focus_box')
    def _check_focus_box(self, proposal):
        value = proposal['value']
        if value[0] >= value[2] or value[1] >= value[3]: 
            raise traitlets.TraitError(f'focus_box invalid: {value}')
        return value
                         
    @traitlets.validate('black_box')
    def _check_black_box(self, proposal):
        value = proposal['value']
        if value[0] > value[2] or value[1] > value[3]: 
            raise traitlets.TraitError(f'Parameter focus_box is invalid: {value}')
        return value
    
    def __str__(self):
        str_ = f'_area_thresh: {self._area_thresh} _focus_box: {self._focus_box} _black_box: {self._black_box} '
        str_ += f'_stride: {self._stride}'
        return str_

In [36]:
from enum import IntEnum, unique

class Test(IntEnum):
    A = 1

In [37]:
a = Test.A

In [38]:
if 1 == a.A:
    print('ok')

ok


### Control Class

In [22]:
class JetRep(traitlets.HasTraits):
    gst_cmd = traitlets.Unicode('gst-launch-1.0', read_only=True)
    shm_path = traitlets.Unicode('/tmp/gst_repnet.shm', read_only=True)
    
    trt_weigth = traitlets.Unicode('/home/nano/models/repnet_b1.trt') 
    trt_msg_timeout = traitlets.Int(5)
    trycheck_count = traitlets.Int(6)
    
    def __init__(self, logger, *args, **kwargs):
        super(JetRep, self).__init__(*args, **kwargs)
        
        self.logger = logger
        self.gst_process = None
        self.engine_process = None
        self.pre_worker = None 
        self.post_worker = None
        self.msg_in = None
        self.msg_out = None
        self.msg_status = None
        
    def start_gst_launch(self):
        # TODO read config, make gst string
        # gst_data_src = CSICamera()
        gst_data_src = USBCamera()
        
        _ = ShareMemorySink(parent=gst_data_src, path=self.shm_path, size=20000000)
        # gst_h264_cvt = H264CodecCvt(gst_data_src)    
        # _ = SRSRtmpSink(parent=gst_h264_cvt, server='172.16.0.35', stream='48b02d5bcfa5')
        # _ = MultiFilesSink(parent=gst_h264_cvt, duration=300, maxfiles=10, location='/home/nano/omega/gst')
        
        # start
        # sudo systemctl restart nvargus-daemon
        self.gst_process = Popen(f'{self.gst_cmd} -e {gst_data_src.gst_str()}', stderr=PIPE, universal_newlines=True, shell=True, start_new_session=True)
        self.logger.info(f'Start gst: {gst_data_src.gst_str()}')
        try:
            _, stderr = self.gst_process.communicate(timeout=3)
            raise RuntimeError(f'error: gst command line[{gst_data_src.gst_str()}]')
        except TimeoutExpired:
            pass
        
        self.logger.debug('Gst-launch started.')


    def stop_gst_launch(self):
        for pid in get_pids(self.gst_cmd):
            # os.killpg(os.getpgid(pid), signal.SIGINT) # start_new_session must be True
            kill_tree(os.getpgid(pid))
        self.gst_process = None


    def start_infer_engine(self):
        if not os.path.exists(self.trt_weigth):
            raise FileNotFoundError(f'error: {self.trt_weigth} is not found')
        self.engine_process = Process(target=repnet_inference_process, args=(
                self.trt_weigth, self.msg_in, self.msg_out, self.msg_status, self.trt_msg_timeout))
        self.engine_process.start()
        for i in range(1, self.trycheck_count + 1):
            try:
                errcode, msg = self.msg_status.get(timeout=self.trt_msg_timeout)
                self.logger.debug(f'start infer engine: {errcode}: {msg}')
                if errcode == 101:
                    break
            except queue.Empty:
                self.logger.debug(f'warning[{i}]: cannot start infer engine !!!')

    
    def stop_infer_engine(self):
        if self.engine_process:
            for i in range(1, self.trycheck_count + 1):
                self.msg_in.put((-1, None))
                try:
                    errcode, msg = self.msg_status.get(timeout=self.trt_msg_timeout)
                    self.logger.debug(f'stop infer engine: {errcode}: {msg}')
                    break
                except queue.Empty:
                    self.logger.debug(f'warning[{i}]: cannot stop infer engine !!!')
        self.engine_process = None

 
    def start_prerep_worker(self):
        self.pre_worker = threading.Thread(target=prerep_process_worker, args=(self.app, self.shm_path))
        self.pre_worker.start()
        self.app.prerep = True

 
    def stop_prerep_worker(self):
        if self.pre_worker:
            self.app.prerep = False
            for i in range(1, self.trycheck_count + 1):
                if not self.pre_worker.is_alive():
                    self.pre_worker.join()
                    break
                time.sleep(0.5)
        self.pre_worker = None


    def start_postrep_worker(self):
        self.post_worker = threading.Thread(target=postrep_process_worker, args=(self.app, ))
        self.post_worker.start()
        self.app.postrep = True

        
    def stop_postrep_worker(self):
        if self.post_worker:
            self.app.postrep = False
            for i in range(1, self.trycheck_count + 1):
                if not self.post_worker.is_alive():
                    self.post_worker.join()
                    break
                time.sleep(0.5)
        self.post_worker = None

        
    def start(self):
        self.stop()
        
        self.msg_in = Queue()
        self.msg_out = Queue()
        self.msg_status = Queue()
        
        # self.start_gst_launch()
        self.start_infer_engine()
        

        self.app = ProcessContext(self.logger, self.msg_in, self.msg_out)
        
        self.start_prerep_worker()
        self.start_postrep_worker()

    def stop(self):
        self.stop_prerep_worker()
        self.stop_postrep_worker()
        self.stop_infer_engine()
        # self.stop_gst_launch()
        
        if self.msg_in: self.msg_in.close()
        if self.msg_out: self.msg_out.close()
        if self.msg_status: self.msg_status.close()

In [23]:
jr = JetRep(logger)

In [24]:
jr.start()

2021-11-11 15:05:13,983 - start_infer_engine:60 - JetRep - DEBUG - start infer engine: 100: starting
2021-11-11 15:05:18,992 - start_infer_engine:64 - JetRep - DEBUG - warning[2]: cannot start infer engine !!!
2021-11-11 15:05:19,612 - start_infer_engine:60 - JetRep - DEBUG - start infer engine: 101: running
2021-11-11 15:05:19,616 - prerep_process_worker:2 - JetRep - DEBUG - Start prerep_process_worker...
2021-11-11 15:05:19,617 - postrep_process_worker:2 - JetRep - DEBUG - Start postrep_process_worker...
2021-11-11 15:05:19,621 - prerep_process_worker:9 - JetRep - DEBUG - Open cammera: [shmsrc socket-path=/tmp/gst_repnet.shm ! video/x-raw, format=BGR,width=640,height=480,framerate=30/1 ! appsink drop=1]


In [25]:
jr.pre_worker.is_alive(),  jr.post_worker.is_alive(), jr.engine_process.is_alive()

(True, True, True)

In [26]:
!pstree -p 11771

2021-11-11 15:05:19,733 - prerep_process_worker:22 - JetRep - DEBUG - Frame info: 30 (640, 480)


In [27]:
# !pstree -p {jr.gst_process.pid}

In [28]:
# !pstree -p {jr.engine_process.pid}

## References

1. [multiprocessing fork() vs spawn()][1] 
<div class="alert alert-info"><code>
import multiprocessing as mp
mp.set_start_method('spawn', force=True)
mp.set_start_method('fork', force=True)
</code></div>
- [traitlets application example][2]
- [metaclasses in python3][3]

[3]: https://stackoverflow.com/questions/100003/what-are-metaclasses-in-python
[2]: https://github.com/cta-observatory/ctapipe/blob/5ff5401af455d91bb76eaf62cf5a15b5545b8d0e/ctapipe/core/tool.py
[1]: https://stackoverflow.com/questions/64095876/multiprocessing-fork-vs-spawn


## FAQs

1. [Failed to create CaptureSession][1]
<div class="alert alert-warning">
This may happen if there is another session using this same camera (hung or alive) and try to use same camera again; if this issue is happening sometimes. You can close all sessions and reset the nvargus-daemon to reset the argus framework and try nvgstcapture-1.0 again. 
<code>
sudo systemctl restart nvargus-daemon
</code></div>

- [Camera remains busy after being released by OpenCV?][2]
<div class="alert alert-warning"><code>
sudo su
systemctl stop nvargus-daemon.service
export enableCamPclLogs=5
export enableCamScfLogs=5
export enableCamCaptureLogs=5
/usr/sbin/nvargus-daemon
</code></div>

- [GStreamer pipeline not cleans up if started as a service][3]

[3]: https://forums.developer.nvidia.com/t/gstreamer-pipeline-not-cleans-up-if-started-as-a-service/184618
[2]: https://forums.developer.nvidia.com/t/camera-remains-busy-after-being-released-by-opencv/173644
[1]: https://forums.developer.nvidia.com/t/error-generated-gstnvarguscamerasrc-cpp-execute-543-failed-to-create-capturesession/112431
